In [11]:
import pandas as pd
import numpy as np
import os
from datetime import datetime, timedelta

# Loop file JSON on folder Time_ETL => Create a DataFrame
dataframes = []
for file in os.listdir(r"D:\python\test\Time_ETL"):
    if file.endswith(".json") :
    # if file.endswith(".json") and not file.startswith('OCB'):
        df = pd.read_json(file)
        dataframes.append(df)
combined_df = pd.concat(dataframes, ignore_index=True)

# Delete a few columns and sort them
COLUMNS_DEL = ['createdDate','executableType','id','invocationID','queueName', 'totalRowsConsumed', 'totalRowsProduced']
COLUMNS_LOCATION = ['name', 'minorRunStatus', 'runStartDate', 'runEndDate', 'runElapsed']
combined_df.drop(columns=COLUMNS_DEL, inplace=True)
combined_df = combined_df[COLUMNS_LOCATION]

# Convert GMT time (GTM +7)
combined_df['runStartDate'] = pd.to_datetime(combined_df['runStartDate']).apply(lambda x : x + timedelta(hours=7))
combined_df['runEndDate'] = pd.to_datetime(combined_df['runEndDate']).apply(lambda x : x + timedelta(hours=7))

# Calculate Elapsed Time
combined_df['runElapsed'] = combined_df['runEndDate'] - combined_df['runStartDate']

# Create new column and extract date and time
combined_df['runDate'] = combined_df['runStartDate'].dt.date
combined_df['runStartTime'] = combined_df['runStartDate'].dt.time
combined_df['runEndTime'] = combined_df['runEndDate'].dt.time

# Group by
combined_df = combined_df.groupby(['name', 'runDate']).agg({'runStartDate': 'min', 'runEndDate': 'max','runElapsed': 'sum', 'runStartTime': "min", 'runEndTime': "max"}).sort_values(by=['name', 'runDate'], ascending=[True, False])

# Convert deltatime on column "runElapsed" to format "HH:mm:ss"
combined_df['runElapsed'] = combined_df['runElapsed'].apply(lambda x : f"{x.days + x.seconds//3600 :02d}:{(x.seconds // 60 ) % 60 :02d}:{x.seconds % 60 :02d}")

# Convert timezone for export file Excel
combined_df['runStartDate'] = combined_df['runStartDate'].dt.tz_localize(None)
combined_df['runEndDate'] = combined_df['runEndDate'].dt.tz_localize(None)

# Reset index (Because after performing a groupby operation on a Pandas DataFrame, the column(s) you used for grouping by become the index of the resulting grouped DataFrame, )
combined_df = combined_df.reset_index()

# Export file
# combined_df.to_excel('date_etl.xlsx', index=False)

In [26]:
import pandas as pd
import numpy as np
import os
from datetime import datetime, timedelta

# Loop file JSON on folder Time_ETL => Create a DataFrame
dataframes = []
for file in os.listdir(r"D:\python\test\Time_ETL"):
    if file.endswith(".json") :
    # if file.endswith(".json") and not file.startswith('OCB'):
        df = pd.read_json(file)
        dataframes.append(df)
combined_df = pd.concat(dataframes, ignore_index=True)

# Delete a few columns and sort them
COLUMNS_DEL = ['createdDate','executableType','id','invocationID','queueName', 'totalRowsConsumed', 'totalRowsProduced']
COLUMNS_LOCATION = ['name', 'minorRunStatus', 'runStartDate', 'runEndDate', 'runElapsed']
combined_df.drop(columns=COLUMNS_DEL, inplace=True)
combined_df = combined_df[COLUMNS_LOCATION]

# Convert GMT time (GTM +7)
combined_df['runStartDate'] = pd.to_datetime(combined_df['runStartDate']).apply(lambda x : x + timedelta(hours=7))
combined_df['runEndDate'] = pd.to_datetime(combined_df['runEndDate']).apply(lambda x : x + timedelta(hours=7))

# Calculate Elapsed Time
# combined_df['runElapsed'] = combined_df['runEndDate'] - combined_df['runStartDate']


In [27]:
combined_df['runElapsed'] = combined_df['runElapsed'].apply(lambda x : int(x/1000))
combined_df['runElapsed'] = combined_df['runElapsed'].apply(lambda x : f"{x//3600 :02d}:{((x % 3600) // 60 ):02d}:{x % 60 :02d}")

In [33]:
# Create new column and extract date and time
combined_df['runDate'] = combined_df['runStartDate'].dt.date
combined_df['runStartTime'] = combined_df['runStartDate'].dt.time
combined_df['runEndTime'] = combined_df['runEndDate'].dt.time

In [24]:
seconds = 24188

hours = seconds // 3600
seconds = seconds % 3600
minutes = seconds // 60 
seconds = seconds % 60

print("{:02d}:{:02d}:{:02d}".format(hours, minutes, seconds))

06:43:08


In [34]:
combined_df

,name,minorRunStatus,runStartDate,runEndDate,runElapsed,runDate,runStartTime,runEndTime
0,OCBDW00SQ00,FWW,2023-12-10 10:39:15+00:00,2023-12-10 17:22:23+00:00,06:43:08,2023-12-10,10:39:15,17:22:23
1,OCBDW00SQ00,FWW,2023-12-09 06:51:35+00:00,2023-12-09 14:06:48+00:00,07:15:13,2023-12-09,06:51:35,14:06:48
2,OCBDW00SQ00,FWW,2023-12-08 06:57:17+00:00,2023-12-08 14:55:14+00:00,07:57:57,2023-12-08,06:57:17,14:55:14
3,OCBDW00SQ00,FWW,2023-12-07 09:49:15+00:00,2023-12-07 18:09:08+00:00,08:19:53,2023-12-07,09:49:15,18:09:08
4,OCBDW00SQ00,FWW,2023-12-06 07:12:19+00:00,2023-12-06 15:59:11+00:00,08:46:52,2023-12-06,07:12:19,15:59:11
...,...,...,...,...,...,...,...,...
26511,OCBDW00SQ18_EOD_EOM,FOK,2020-08-20 15:56:56+00:00,2020-08-20 15:57:13+00:00,00:00:17,2020-08-20,15:56:56,15:57:13
26512,OCBDW00SQ18_EOD_EOM,FOK,2020-08-19 12:56:41+00:00,2020-08-19 12:56:56+00:00,00:00:15,2020-08-19,12:56:41,12:56:56
26513,OCBDW00SQ18_EOD_EOM,FOK,2020-08-18 12:48:03+00:00,2020-08-18 12:48:17+00:00,00:00:14,2020-08-18,12:48:03,12:48:17
26514,OCBDW00SQ18_EOD_EOM,FOK,2020-08-16 19:35:18+00:00,2020-08-16 19:35:34+00:00,00:00:16,2020-08-16,19:35:18,19:35:34


In [7]:
combined_df[(combined_df['runStartDate'].dt.date == pd.to_datetime('2023-09-11').date()) & (combined_df['name'] == 'OCBDW00SQ00')]

,name,minorRunStatus,runStartDate,runEndDate,runElapsed
229,OCBDW00SQ00,FWW,2023-09-11 09:08:10+00:00,2023-09-11 09:40:51+00:00,1961000
230,OCBDW00SQ00,FWF,2023-09-11 08:17:27+00:00,2023-09-11 08:58:45+00:00,2478000


In [37]:
# Create new column and extract date and time
combined_df['runDate'] = combined_df['runStartDate'].dt.date
combined_df['runStartTime'] = combined_df['runStartDate'].dt.time
combined_df['runEndTime'] = combined_df['runEndDate'].dt.time


# Convert timezone for export file Excel
combined_df['runStartDate'] = combined_df['runStartDate'].dt.tz_localize(None)
combined_df['runEndDate'] = combined_df['runEndDate'].dt.tz_localize(None)

In [10]:
combined_df[(combined_df['runStartDate'].dt.date >= pd.to_datetime('2023-09-01').date()) & (combined_df['name'] == 'OCBDW00SQ00')].sort_values(by=['name', 'runStartDate'], ascending=[False, True]).head(30)

,name,minorRunStatus,runStartDate,runEndDate,runElapsed
253,OCBDW00SQ00,FWW,2023-09-01 08:38:33+00:00,2023-09-02 03:36:06+00:00,68253000
252,OCBDW00SQ00,FWF,2023-09-06 09:40:37+00:00,2023-09-07 03:35:09+00:00,64472000
251,OCBDW00SQ00,FWF,2023-09-07 06:26:43+00:00,2023-09-07 06:50:07+00:00,1404000
250,OCBDW00SQ00,FWF,2023-09-07 06:56:14+00:00,2023-09-07 07:17:22+00:00,1268000
249,OCBDW00SQ00,FWF,2023-09-07 07:22:21+00:00,2023-09-07 07:41:25+00:00,1144000
248,OCBDW00SQ00,FWF,2023-09-07 07:42:13+00:00,2023-09-07 07:59:47+00:00,1054000
247,OCBDW00SQ00,FWF,2023-09-07 08:15:01+00:00,2023-09-07 08:31:09+00:00,968000
246,OCBDW00SQ00,FWF,2023-09-07 08:31:49+00:00,2023-09-07 08:45:25+00:00,816000
245,OCBDW00SQ00,FWF,2023-09-07 08:49:46+00:00,2023-09-07 09:38:45+00:00,2939000
244,OCBDW00SQ00,FWF,2023-09-07 09:47:39+00:00,2023-09-07 10:47:17+00:00,3578000


In [59]:
combined_df.sort_values(by=['name', 'runStartDate'], ascending=[False, False]).head(20)

,name,minorRunStatus,runStartDate,runEndDate,runElapsed
25470,OCBDW00SQ18_EOD_EOM,FOK,2023-12-08 14:49:08+00:00,2023-12-08 14:55:14+00:00,0 days 00:06:06
25471,OCBDW00SQ18_EOD_EOM,FOK,2023-12-07 18:02:53+00:00,2023-12-07 18:09:08+00:00,0 days 00:06:15
25472,OCBDW00SQ18_EOD_EOM,FOK,2023-12-06 15:53:25+00:00,2023-12-06 15:59:11+00:00,0 days 00:05:46
25473,OCBDW00SQ18_EOD_EOM,FOK,2023-12-05 16:02:12+00:00,2023-12-05 16:08:15+00:00,0 days 00:06:03
25474,OCBDW00SQ18_EOD_EOM,FOK,2023-12-03 16:51:03+00:00,2023-12-03 17:00:07+00:00,0 days 00:09:04
25475,OCBDW00SQ18_EOD_EOM,FOK,2023-12-02 16:57:10+00:00,2023-12-02 17:03:35+00:00,0 days 00:06:25
25476,OCBDW00SQ18_EOD_EOM,FWW,2023-12-02 08:30:51+00:00,2023-12-02 09:37:27+00:00,0 days 01:06:36
25477,OCBDW00SQ18_EOD_EOM,FWF,2023-12-01 22:55:40+00:00,2023-12-01 22:55:53+00:00,0 days 00:00:13
25478,OCBDW00SQ18_EOD_EOM,FWF,2023-12-01 19:02:02+00:00,2023-12-01 20:25:39+00:00,0 days 01:23:37
25479,OCBDW00SQ18_EOD_EOM,FOK,2023-11-30 15:32:17+00:00,2023-11-30 15:38:37+00:00,0 days 00:06:20


In [48]:
combined_df['minorRunStatus'].value_counts()

FWW    20160
FWF     5152
FOK     1111
STP       31
SYN       10
RNW        2
Name: minorRunStatus, dtype: int64

In [35]:
import datetime

start = '20230901'
end = '20231210'
start_date = datetime.date(int(start[:4]), int(start[4:6]), int(start[6:]))
end_date = datetime.date(int(end[:4]), int(end[4:6]), int(end[6:]))

condition = []
while start_date <= end_date:
    delta = datetime.timedelta(days=1)
    curDay =  start_date.strftime('%Y-%m-%d')
    # curDay =  start_date.strftime('%d-%b-%y')
    start_date += delta
    condition.append(curDay)

In [ ]:
condition

In [38]:
# condition = ['2023-09-01', '2023-12-07']
combined_df[combined_df['runDate'].astype(str).isin(condition)].to_excel(f'date_etl_{condition[0]}.xlsx', index=False)

In [27]:
combined_df[combined_df['runDate'].astype(str).isin(condition)].to_excel('date_etl_SUB_JOB_20231108.xlsx', index=False)

In [6]:
combined_df[combined_df['runDate'].astype(str) == '2023-11-08'].to_excel('date_etl_20231108.xlsx', index=False)

In [4]:
combined_df[(combined_df['runDate'].astype(str) < '2023-09-01') & (combined_df['runDate'].astype(str) == '2023-12-07')].to_excel('date_etl_SUB_JOB_20231208.xlsx', index=False)

In [4]:
BEGINDATE = 20221201
ENDDATE = 20221207

while BEGINDATE <= ENDDATE:

    SQL = f"SELECT * FROM OCBDW2.WORKING_DATE WHERE  TO_CHAR(MSR_PRD_DT, 'yyyymmdd') = '{BEGINDATE}'"

    print(SQL)

    BEGINDATE +=1

SELECT * FROM OCBDW2.WORKING_DATE WHERE  TO_CHAR(MSR_PRD_DT, 'yyyymmdd') = '20221201'
SELECT * FROM OCBDW2.WORKING_DATE WHERE  TO_CHAR(MSR_PRD_DT, 'yyyymmdd') = '20221202'
SELECT * FROM OCBDW2.WORKING_DATE WHERE  TO_CHAR(MSR_PRD_DT, 'yyyymmdd') = '20221203'
SELECT * FROM OCBDW2.WORKING_DATE WHERE  TO_CHAR(MSR_PRD_DT, 'yyyymmdd') = '20221204'
SELECT * FROM OCBDW2.WORKING_DATE WHERE  TO_CHAR(MSR_PRD_DT, 'yyyymmdd') = '20221205'
SELECT * FROM OCBDW2.WORKING_DATE WHERE  TO_CHAR(MSR_PRD_DT, 'yyyymmdd') = '20221206'
SELECT * FROM OCBDW2.WORKING_DATE WHERE  TO_CHAR(MSR_PRD_DT, 'yyyymmdd') = '20221207'
